In [1]:
!pip install opencv-python
!pip install deepface
!pip install face_recognition


In [2]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


Inefficiencies in the code: : The emotions do not map other that neutral, check the emoji documentation.
Second : Learn how to improve this model. 


In [3]:
import cv2
from deepface import DeepFace
import face_recognition
import time
import emoji

# Mapping emotions to emojis
def map_to_emoji(emotion):
    emoji_mapping = {
        "Happiness": emoji.emojize(":smile:"),
        "Sadness": emoji.emojize(":cry:"),
        "Anger": emoji.emojize(":angry:"),
        "Fear": emoji.emojize(":scream:"),
        "Disgust": emoji.emojize(":nauseated_face:"),
        "Surprise": emoji.emojize(":open_mouth:"),
        "Neutral": emoji.emojize(":neutral_face:")
    }
    return emoji_mapping.get(emotion, emoji.emojize(":question:"))  # Use a question mark emoji for unknown emotions


def map_to_basic_emotion(emotion):
    basic_emotions = {
        "happy": "Happiness",
        "sad":   "Sadness",
        "angry": "Anger",
        "fear":  "Fear",
        "disgust": "Disgust",
        "surprise": "Surprise",
        "neutral": "Neutral"
    }
    return basic_emotions.get(emotion, "Unknown")

# Initializes the webcam
cap = cv2.VideoCapture(0)

start_time = time.time()
DURATION = 5  # time can be given in seconds

results = []  # To store the results

while True:
    elapsed_time = time.time() - start_time
    if elapsed_time > DURATION:
        break

    ret, frame = cap.read()
    if not ret:
        break

    face_locations = face_recognition.face_locations(frame)

    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = frame[top:bottom, left:right]
        predictions = DeepFace.analyze(face_image, actions=['emotion'], enforce_detection=False)

        # Checking for the type of predictions and extract dominant emotion
        if isinstance(predictions, dict):
            raw_emotion = predictions.get('dominant_emotion', 'Unknown')
        elif isinstance(predictions, list) and len(predictions) > 0:
            raw_emotion = predictions[0].get('dominant_emotion', 'Unknown')
        else:
            raw_emotion = "Unknown"

        basic_emotion = map_to_basic_emotion(raw_emotion)
        
        # Convert emotion to emoji
        emo = map_to_emoji(basic_emotion)
        
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        results.append((timestamp, basic_emotion, emo))

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, basic_emotion, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    cv2.imshow('Emotion Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

with open("emotion_results.txt", "w") as file:
    file.write("Timestamp\tEmotion\tEmoji\n")
    for r in results:
        file.write(f"{r[0]}\t{r[1]}\t{r[2]}\n")


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 34.57it/s]
